### This notebook is to validate the population data between the WNV dataset (that was initially provided by Dr.Aiman) and the population data that we have got from the ACS census data

In [171]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

Importing the data:

In [172]:
wnv = pd.read_csv('wnv_aggr_final.csv') #The WNV data given by Dr.Aiman
se = pd.read_csv('se_data.csv') #The Socio-Economic data that was cleaned by Vinu

In [173]:
#Getting the idea of the wnv dataset
print(wnv.columns)
print("\nShape: ", wnv.shape)
print("\nYears: ", wnv.YEAR.unique())
print("\nCounties: ", len(wnv.GEOID10.unique()))


#Taking only the columns needed for the validation
wnv_sample = wnv[["YEAR", "GEOID10", "POPESTIMATE"]]
print("\nSample shape: ", wnv_sample.shape)

Index(['YEAR', 'GEOID10', 'CTYNAME', 'STUSPS', 'DeG12Region', 'Total_BG.sBG',
       'County_WNV_Class', 'County_WNV_Class2', 'POPESTIMATE', 'NI_', 'NNI_',
       'NI_IR_', 'Establish_', 'NIIR_Z', 'TEMPERATURE', 'HUMIDITY',
       'PRECIPITATION', 'CLIMATE_ZONE'],
      dtype='object')

Shape:  (14484, 18)

Years:  [1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015]

Counties:  852

Sample shape:  (14484, 3)


In [174]:
#This function takes the dataframe and the year for which we want to compare and returns the yearly dataframe
#### WNV dataset

def get_year_data_wnv(df, yr):
    
    data = df[df["YEAR"] == yr]
    print("\nWNV", yr, "shape: ", data.shape)
    print("\n", data.columns)
    return data

In [175]:
#Getting the idea of the Socio-Economic dataset

print(se.columns)
print("\nShape: ", se.shape)
print("\nYears: ", se.YEAR.unique())
print("\nCounties: ", len(se.STCOU.unique()))

Index(['YEAR', 'Areaname', 'STCOU', 'Median_Household_Income',
       'Poverty_number_of_people', 'Poverty_percent_of_people',
       'Public_school_enrollment_Fall'],
      dtype='object')

Shape:  (44772, 7)

Years:  [1995 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009]

Counties:  3198


In [176]:
#Taking only the columns needed for the validation

se_sample = se[["YEAR", "STCOU", "Poverty_number_of_people", "Poverty_percent_of_people"]]
se_sample["Population"] = (se_sample["Poverty_number_of_people"] * 100)/se_sample["Poverty_percent_of_people"]
pd.options.display.float_format = '{:.2f}'.format
print("Sample shape: ", se_sample.shape)
se_sample.head()

Sample shape:  (44772, 5)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,YEAR,STCOU,Poverty_number_of_people,Poverty_percent_of_people,Population
0,1995,0,36424609,13.80,263946442.03
1,1995,1000,752097,17.60,4273278.41
2,1995,1001,5471,13.40,40828.36
3,1995,1003,15494,12.40,124951.61
4,1995,1005,6423,24.90,25795.18


In [177]:
#This function takes the dataframe and the year for which we want to compare and returns the yearly dataframe
#### Socio-Economic dataset

def get_year_data_se(df, yr):
    
    data = df[df["YEAR"] == yr]
    print("\nSocio-Economic ", yr ,"shape: ", data.shape)
    print("\n", data.columns)
    return data

In [178]:
#Function to merge the WNV and Socio-Economic dataframes for a specific year

def get_combined_df(df1, df2):

    combine = pd.merge(df1, df2, left_on='GEOID10', right_on='STCOU', how='inner')
    combine = combine[["YEAR_x", "GEOID10", "POPESTIMATE", "Population", "Poverty_number_of_people", 
                   "Poverty_percent_of_people"]]
    combine.rename(columns = {'YEAR_x':'Year', 'GEOID10':'Geoid', 'POPESTIMATE':'Population_wnv', 
                          'Population':'Calculated_Population_se'  }, inplace = True) 
    print("Combined data shape: ", combine.shape)
    return combine

In [179]:
#Function to plot GEOID v/s population from both datasets

def plot_graph(df):

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=combine.Geoid, y=combine.Population_wnv, name='Pop WNV file',
                         line=dict(color='firebrick', width=2)))
    fig.add_trace(go.Scatter(x=combine.Geoid, y=combine.Calculated_Population_se, name='Pop SE file',
                         line=dict(color='royalblue', width=2)))
    return fig

Main function below:

In [180]:
wnv_yr = get_year_data_wnv(wnv_sample, 2000)  #Checking for the year 2000 
#### (Please replace 2000 with the year you want to check for)
print("-----------------------------------------------------------")
se_yr = get_year_data_se(se_sample, 2000)  #Checking for the year 2000 


WNV 2000 shape:  (852, 3)

 Index(['YEAR', 'GEOID10', 'POPESTIMATE'], dtype='object')
-----------------------------------------------------------

Socio-Economic  2000 shape:  (3198, 5)

 Index(['YEAR', 'STCOU', 'Poverty_number_of_people',
       'Poverty_percent_of_people', 'Population'],
      dtype='object')


In [181]:
# Combining the WNV and Socio-Economic dataframes into one

combined_data = get_combined_df(wnv_yr, se_yr)
combined_data.head(5)

Combined data shape:  (852, 6)


,Year,Geoid,Population_wnv,Calculated_Population_se,Poverty_number_of_people,Poverty_percent_of_people
0,2000,1001,44021,44580.95,4681,10.50
1,2000,1003,141342,142835.05,13855,9.70
2,2000,1015,111081,109675.50,16561,15.10
3,2000,1033,54997,54412.21,7128,13.10
4,2000,1045,49124,47457.52,7261,15.30


In [182]:
plot_graph(combined_data)

#### The population for the year 2000 from both the datasets are very close as we can barely see the difference between the two lines
#### Please zoom on the graph to see the two lines

Checking the error and percentage error:

In [183]:
combined_errors = combined_data[['Geoid', 'Population_wnv', 'Calculated_Population_se']]
combined_errors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 852 entries, 0 to 851
Data columns (total 3 columns):
Geoid                       852 non-null int64
Population_wnv              852 non-null object
Calculated_Population_se    851 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 26.6+ KB


In [184]:
# Sorting the dataframe to see null/invalid values
combined_errors.sort_values(by=['Calculated_Population_se'], ascending=False).tail(5)

,Geoid,Population_wnv,Calculated_Population_se
782,51685,10305,10864.41
775,51610,10441,10689.66
147,13053,15047,10395.16
796,51830,12012,7412.43
81,8014,39679,nan


In [185]:
## Run this block only if the above output has nan, inf or blank values
## Replace 81 with the index number of the row containing the nan, inf or blank values

## DO NOT run this block multiple times

combined_errors = combined_errors.drop([81])
combined_errors.reset_index(drop=True, inplace=True)
print("After dropping the invalid row: ", combined_errors.shape)
combined_errors = combined_errors.apply(pd.to_numeric)
print(combined_errors.dtypes)
combined_errors["Error"] = (combined_errors["Population_wnv"] - combined_errors["Calculated_Population_se"]).abs()
combined_errors.head(5)


After dropping the invalid row:  (851, 3)
Geoid                         int64
Population_wnv                int64
Calculated_Population_se    float64
dtype: object


,Geoid,Population_wnv,Calculated_Population_se,Error
0,1001,44021,44580.95,559.95
1,1003,141342,142835.05,1493.05
2,1015,111081,109675.50,1405.50
3,1033,54997,54412.21,584.79
4,1045,49124,47457.52,1666.48


In [186]:
#Checking the values with greater than 10% error

combined_errors["Percentage_error"] = (combined_errors["Error"]/combined_errors["Population_wnv"])*100
print("Values with greater than 10% error: \n\n",combined_errors[combined_errors.Percentage_error > 10.0 ].count())

#Sorting by percentage error
combined_errors.sort_values(by=['Percentage_error'], ascending=False)

Values with greater than 10% error: 

 Geoid                       22
Population_wnv              22
Calculated_Population_se    22
Error                       22
Percentage_error            22
dtype: int64


,Geoid,Population_wnv,Calculated_Population_se,Error,Percentage_error
795,51830,12012,7412.43,4599.57,38.29
146,13053,15047,10395.16,4651.84,30.92
691,48099,75199,57949.64,17249.36,22.94
787,51750,15872,12889.53,2982.47,18.79
777,51660,40437,33619.29,6817.71,16.86
...,...,...,...,...,...
221,18011,46390,46377.36,12.64,0.03
226,18039,183412,183457.83,45.83,0.02
238,18109,66838,66830.99,7.01,0.01
579,41029,181775,181765.62,9.38,0.01
